In [59]:
using Distributions
using ForwardDiff
using StatsFuns

In [60]:
# generate date
n = 100
μ_true = 0.4
σ_true = 0.2
X = rand(Normal(μ_true, σ_true), n)

prior = Normal(0, 1)

Distributions.Normal{Float64}(μ=0.0, σ=1.0)

In [61]:
function ℒ(X, prior, ϕ, L)
    ϵ = rand(Normal(), L)
    z = ϕ[1] + softplus(ϕ[2]) * ϵ
    
    log_prior = logpdf(prior, z)
    log_lik = map(param -> loglikelihood(Normal(param, σ_true), X), z)
    log_joint = log_prior + log_lik
    
    entropy = logpdf(Normal(ϕ[1], softplus(ϕ[2])), z)
    
    return mean(log_joint - entropy)
end

ℒ (generic function with 1 method)

In [68]:
# gradient descent parameters
ϵ = 1e-8
G_t = zeros(2)
η = 1.0

# number of epochs
epochs = 100

# initialize ϕ
ϕ = rand(Normal(), 2)
ϕ[2] = softplus(ϕ[2])

# number of monte carlo simulations per epoch
L = 10

for i in 1:epochs
    g = ForwardDiff.gradient(γ -> ℒ(X, prior, γ, L), ϕ)
    G_t += (g .^ 2)
    δ = (η ./ sqrt.(G_t + ϵ)) .* g
    ϕ += δ
end

println([ϕ[1], softplus(ϕ[2])])

[0.411357, 0.0282846]


In [63]:
μ₀ = params(prior)[1]
σ₀² = params(prior)[2]^2

post_var = 1 / (1 / σ₀² + n / (σ_true^2))
post_mean = (μ₀ / σ₀² + sum(X) / (σ_true^2)) * post_var
ϕ = [post_mean, sqrt(post_var)]

println(ϕ)

[0.418692, 0.019996]


In [56]:
softplus(ϕ[2])

0.06977082502055569

In [58]:
μ₀ = params(prior)[1]
σ₀² = params(prior)[2]^2

post_var = 1 / (1 / σ₀² + n / (σ_true^2))
post_mean = (μ₀ / σ₀² + sum(X) / (σ_true^2)) * post_var
println(post_mean)
println(post_var)

0.3987794344143785
3.9998400063997445e-5


In [47]:
η ./ sqrt.(G_t + ϵ)

2-element Array{Float64,1}:
 5.59367e-7
 3.51887e-7

In [24]:
loglikelihood(Normal(z[2], σ_true), X)

232.10289437160029

In [43]:
ϕ

2-element Array{Float64,1}:
 0.39781   
 0.00605403